In [1]:
%load_ext remotespark
import requests
requests.packages.urllib3.disable_warnings()

In [3]:
%spark info

Info for running sparkmagic:
    mode=normal
    Possible endpoints are: []



In [3]:
%spark add pys python url=https://livy0.cloudapp.net/livy;username=admin;password=HdpCli123!

Info for running sparkmagic:
    mode=normal
    Possible endpoints are: ['pys']



In [4]:
%%spark -s True -c pys
SHOW TABLES

""


In [5]:
%%spark
hvacText = sc.textFile("wasb://agusparkeight@agushvohrastore.blob.core.windows.net/HdiSamples/SensorSampleData/hvac/HVAC.csv")
hvacSchema = StructType([StructField("date", StringType(), False),StructField("time", StringType(), False),StructField("targettemp", IntegerType(), False),StructField("actualtemp", IntegerType(), False),StructField("buildingID", StringType(), False)])
hvac = hvacText.map(lambda s: s.split(",")).filter(lambda s: s[0] != "Date").map(lambda s:(str(s[0]), str(s[1]), int(s[2]), int(s[3]), str(s[6]) ))
hvacdf = sqlContext.createDataFrame(hvac,hvacSchema)
hvacdf.registerTempTable("hvac")
data = sqlContext.sql("select buildingID, (targettemp - actualtemp) as temp_diff, date from hvac where date = \"6/1/13\"")
data.columns

u"['buildingID', 'temp_diff', 'date']"

In [6]:
%%spark -s True
SHOW TABLES

,isTemporary,tableName
0,True,hvac


In [7]:
%%spark -s True
select buildingID, (targettemp - actualtemp) as temp_diff, date from hvac where date = '6/1/13'

,buildingID,date,temp_diff
0,4,6/1/13,8
1,3,6/1/13,2
2,7,6/1/13,-10
3,12,6/1/13,3
4,7,6/1/13,9
5,7,6/1/13,5
6,3,6/1/13,11
7,8,6/1/13,-7
8,17,6/1/13,14
9,16,6/1/13,-3


# Scala

In [8]:
%spark add scala scala url=https://livy0.cloudapp.net/livy;username=admin;password=HdpCli123!

Info for running sparkmagic:
    mode=normal
    Possible endpoints are: ['pys', 'scala']



In [9]:
%%spark -c scala
import sqlContext.implicits._
val hvacText = sc.textFile("wasb://agusparkeight@agushvohrastore.blob.core.windows.net/HdiSamples/SensorSampleData/hvac/HVAC.csv")

case class Hvac(date: String, time: String, targettemp: Integer, actualtemp: Integer, buildingID: String)
val hvac = hvacText.map(s => s.split(",")).filter(s => s(0) != "Date").map(
    s => Hvac(s(0), 
            s(1),
            s(2).toInt,
            s(3).toInt,
            s(6)
        )
).toDF()
hvac.registerTempTable("hvac")

u'import sqlContext.implicits._\nhvacText: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1] at textFile at <console>:12\ndefined class Hvac\nhvac: org.apache.spark.sql.DataFrame = [date: string, time: string, targettemp: int, actualtemp: int, buildingID: string]'

In [10]:
%%spark -s True -c scala
show tables

,isTemporary,tableName
0,True,hvac


In [11]:
%%spark -c scala
val a = sqlContext.sql("select buildingID, (targettemp - actualtemp) as temp_diff, date from hvac where date = \"6/1/13\"").toJSON.take(3)

u'a: Array[String] = Array({"buildingID":"4","temp_diff":8,"date":"6/1/13"}, {"buildingID":"3","temp_diff":2,"date":"6/1/13"}, {"buildingID":"7","temp_diff":-10,"date":"6/1/13"})'

In [12]:
%%spark -s True -c scala
select buildingID, (targettemp - actualtemp) as temp_diff, date from hvac where date = '6/1/13'

,buildingID,date,temp_diff
0,4,6/1/13,8
1,3,6/1/13,2
2,7,6/1/13,-10
3,12,6/1/13,3
4,7,6/1/13,9
5,7,6/1/13,5
6,3,6/1/13,11
7,8,6/1/13,-7
8,17,6/1/13,14
9,16,6/1/13,-3


# Helpers for direct requests

In [7]:
# Create session
import requests
import json
url = "https://livy0.cloudapp.net/livy/sessions"
headers = {"Content-Type": "application/json"}
username = "admin"
password = "HdpCli123!"
data = {"kind": "pyspark"}
r = requests.post(url, headers=headers, auth=(username, password), data=json.dumps(data), verify=False)
r.text

/Users/alexgg/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


u'{"id":0,"state":"starting","kind":"pyspark","log":[]}'

In [19]:
# Execute code
import requests
import json
url = "https://livy0.cloudapp.net/livy/sessions/1/statements"
headers = {"Content-Type": "application/json"}
username = "admin"
password = "HdpCli123!"
#code = 'val a = sqlContext.sql("select buildingID, (targettemp - actualtemp) as temp_diff, date from hvac")'
code = '%table a'
data = {"code": code}
r = requests.post(url, headers=headers, auth=(username, password), data=json.dumps(data), verify=False)
r.text

/Users/alexgg/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


u'{"id":10,"state":"running","output":null}'

In [20]:
# Get statements
import requests
import json
url = "https://livy0.cloudapp.net/livy/sessions/1/statements/10"
headers = {"Content-Type": "application/json"}
username = "admin"
password = "HdpCli123!"
r = requests.get(url, headers=headers, auth=(username, password), verify=False)
r.text

/Users/alexgg/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


u'{"id":10,"state":"available","output":{"status":"error","execution_count":10,"ename":"Error","evalue":"<console>:20: error: not found: value %\\n              %table a\\n              ^"}}'

# By executing data.toJSON().collect()

In [8]:
r = '[u\'{"buildingID":"4","temp_diff":8,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":14,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":14,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":8,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"4","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":8,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-15,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":8,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":14,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":11,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"4","temp_diff":8,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":14,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"4","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"4","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-10,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"1","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-5,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":7,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":9,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-13,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"12","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-12,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"9","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"10","temp_diff":-3,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":-1,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-4,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"13","temp_diff":3,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":13,"date":"6/1/13"}\', u\'{"buildingID":"14","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":12,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"2","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"17","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"19","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":1,"date":"6/1/13"}\', u\'{"buildingID":"20","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":-11,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":-6,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-9,"date":"6/1/13"}\', u\'{"buildingID":"18","temp_diff":0,"date":"6/1/13"}\', u\'{"buildingID":"3","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"7","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":14,"date":"6/1/13"}\', u\'{"buildingID":"8","temp_diff":6,"date":"6/1/13"}\', u\'{"buildingID":"4","temp_diff":10,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":4,"date":"6/1/13"}\', u\'{"buildingID":"16","temp_diff":-7,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":-2,"date":"6/1/13"}\', u\'{"buildingID":"15","temp_diff":2,"date":"6/1/13"}\', u\'{"buildingID":"6","temp_diff":5,"date":"6/1/13"}\', u\'{"buildingID":"11","temp_diff":-8,"date":"6/1/13"}\', u\'{"buildingID":"5","temp_diff":-10,"date":"6/1/13"}\']'

In [11]:
jsonData = eval(r)

In [12]:
s = "[{}]".format(",".join(jsonData))

In [15]:
import pandas as pd
import json
pd.DataFrame(json.loads(s))

,buildingID,date,temp_diff
0,4,6/1/13,8
1,3,6/1/13,2
2,7,6/1/13,-10
3,12,6/1/13,3
4,7,6/1/13,9
5,7,6/1/13,5
6,3,6/1/13,11
7,8,6/1/13,-7
8,17,6/1/13,14
9,16,6/1/13,-3


# From data.toJSON().take(5) python

In [24]:
resultjsontake = "[u\'{\"buildingID\":\"4\",\"temp_diff\":8,\"date\":\"6/1/13\"}\', u\'{\"buildingID\":\"3\",\"temp_diff\":2,\"date\":\"6/1/13\"}\']"

In [26]:
arrayresultjsontake = eval(resultjsontake)
arrayresultjsontake

[u'{"buildingID":"4","temp_diff":8,"date":"6/1/13"}',
 u'{"buildingID":"3","temp_diff":2,"date":"6/1/13"}']

# From data.toJSON().take(5) scala

In [25]:
resultjsontake = u'res11: Array[String] = Array({"buildingID":"4","temp_diff":8,"date":"6/1/13"}, {"buildingID":"3","temp_diff":2,"date":"6/1/13"}, {"buildingID":"7","temp_diff":-10,"date":"6/1/13"})'

In [49]:
import re
m = re.search('Array\[String\] = Array\((.*)\)', resultjsontake)
re_resultjsontake = m.group(1)
re_resultjsontake

u'{"buildingID":"4","temp_diff":8,"date":"6/1/13"}, {"buildingID":"3","temp_diff":2,"date":"6/1/13"}, {"buildingID":"7","temp_diff":-10,"date":"6/1/13"}'

In [48]:
arrayresultjsontake = eval(re_resultjsontake)
arrayresultjsontake

({'buildingID': '4', 'date': '6/1/13', 'temp_diff': 8},
 {'buildingID': '3', 'date': '6/1/13', 'temp_diff': 2},
 {'buildingID': '7', 'date': '6/1/13', 'temp_diff': -10})

In [8]:
a = 'res2: Array[String] = Array({"tableName":"hvac","isTemporary":true})'
import re
m = re.search('Array\[String\] = Array\((.*)\)', a)
re_resultjsontake = m.group(1)
if re_resultjsontake == "":
    re_resultjsontake = "()"
print(re_resultjsontake)
eval(re_resultjsontake)

{"tableName":"hvac","isTemporary":true}


NameError: name 'true' is not defined

In [10]:
"[{}]".format(",".join(eval(re_resultjsontake)))

'[]'

In [8]:
records_text = '{"buildingID":"1","temp_diff":12,"date":"6/1/13"}\n{"buildingID":"15","temp_diff":0,"date":"6/1/13"}'
jsonArray = "[{}]".format(",".join(records_text.split("\n")))

In [9]:
import pandas as pd
import json
pd.DataFrame(json.loads(jsonArray))

,buildingID,date,temp_diff
0,1,6/1/13,12
1,15,6/1/13,0


In [14]:
a = json.loads(jsonArray)
a

[{u'buildingID': u'1', u'date': u'6/1/13', u'temp_diff': 12},
 {u'buildingID': u'15', u'date': u'6/1/13', u'temp_diff': 0}]